# Strategic Divergence Visualizations - ESPN/ADP Blind Spot Detection

This notebook identifies strategic blind spots where the 80% ESPN + 20% ADP weighting may mislead draft decisions by analyzing ranking divergences.

**Filtered to Top 225 Players Only** - Removes late-round irrelevant players for draft relevance.

In [9]:
# Required imports
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

print("✅ Imports loaded successfully")

✅ Imports loaded successfully


In [10]:
# Load and merge ranking data
def load_and_merge_ranking_data():
    """Load ESPN rankings and external ADP data, merge them for probability calculations"""
    
    # Load ESPN data (80% weight)
    espn_df = pd.read_csv('../data/espn_projections_20250814.csv')
    
    # Load FantasyPros ADP data (20% weight)  
    adp_df = pd.read_csv('../data/fantasypros_adp_20250815.csv')
    
    print(f"ESPN data: {len(espn_df)} players")
    print(f"ADP data: {len(adp_df)} players")
    
    # Standardize player names for matching
    def clean_name(name):
        return str(name).strip().lower().replace("'", "").replace(".", "")
    
    espn_df['clean_name'] = espn_df['player_name'].apply(clean_name)
    adp_df['clean_name'] = adp_df['PLAYER'].apply(clean_name)
    
    # Merge ESPN with ADP data
    merged_df = espn_df.merge(
        adp_df[['clean_name', 'RANK', 'ADP']], 
        on='clean_name', 
        how='left'
    )
    
    # Fill missing ADP ranks with ESPN rank + some penalty
    merged_df['adp_rank'] = merged_df['RANK'].fillna(merged_df['overall_rank'] + 50)
    merged_df['espn_rank'] = merged_df['overall_rank']
    
    print(f"Merged dataset: {len(merged_df)} players")
    print(f"Players with ADP data: {len(merged_df[merged_df['RANK'].notna()])} players")
    
    return merged_df[['overall_rank', 'position', 'position_rank', 'player_name', 'team', 'salary_value', 'bye_week', 'espn_rank', 'adp_rank']]

# Load the data
ranking_df = load_and_merge_ranking_data()
print("\n✅ Ranking data loaded")

ESPN data: 300 players
ADP data: 347 players
Merged dataset: 300 players
Players with ADP data: 252 players

✅ Ranking data loaded


In [11]:
# Core probability functions
def compute_softmax_scores(rank_series, tau=5.0):
    """Convert ranks to softmax scores with temperature tau (lower rank = higher score)"""
    scores = np.exp(-rank_series / tau)
    return scores

def compute_pick_probabilities(available_df, espn_weight=0.8, adp_weight=0.2, tau_espn=5.0, tau_adp=5.0):
    """
    Compute per-player probability of being picked next using weighted softmax over ESPN and ADP ranks
    """
    if len(available_df) == 0:
        return pd.Series(dtype=float)
    
    # Softmax scores for ESPN and ADP ranks
    espn_scores = compute_softmax_scores(available_df['espn_rank'], tau_espn)
    adp_scores = compute_softmax_scores(available_df['adp_rank'], tau_adp)
    
    # Weighted combination (80% ESPN, 20% ADP)
    combined_scores = espn_weight * espn_scores + adp_weight * adp_scores
    
    # Normalize to sum to 1 across available players
    probs = combined_scores / combined_scores.sum()
    return probs

def probability_gone_before_next_pick(available_df, player_name, picks_until_next_turn):
    """
    Compute probability a player is gone before your next turn using discrete survival calculation
    """
    if picks_until_next_turn <= 0:
        return 0.0
    
    survival_prob = 1.0
    current_available = available_df.copy()
    
    # Simulate each pick until our next turn
    for pick_step in range(picks_until_next_turn):
        if len(current_available) == 0:
            break
            
        # Get probabilities for current available players
        pick_probs = compute_pick_probabilities(current_available)
        
        # Find probability our target player gets picked this round
        player_mask = current_available['player_name'] == player_name
        if not player_mask.any():
            # Player already gone
            break
            
        p_pick_now = pick_probs[player_mask].iloc[0] if player_mask.any() else 0.0
        
        # Update survival probability
        survival_prob *= (1 - p_pick_now)
        
        # For simulation: remove the most likely pick (simplified approach)
        most_likely_idx = pick_probs.idxmax()
        current_available = current_available.drop(most_likely_idx)
    
    prob_gone = 1 - survival_prob
    return min(1.0, max(0.0, prob_gone))

print("✅ Core probability functions loaded")

✅ Core probability functions loaded


In [12]:
# Add divergence metrics to dataframe
def add_divergence_metrics(df):
    """Add divergence calculations to existing dataframe"""
    df = df.copy()
    df['rank_divergence'] = df['espn_rank'] - df['adp_rank']
    df['abs_divergence'] = abs(df['rank_divergence'])
    
    # Add availability probabilities for analysis
    my_picks = [8, 17, 32, 41, 56, 65, 80, 89]
    current_pick = 1
    next_pick = min([p for p in my_picks if p > current_pick])
    picks_until = next_pick - current_pick
    
    availability_probs = []
    for _, player in df.iterrows():
        prob_gone = probability_gone_before_next_pick(df, player['player_name'], picks_until)
        availability_probs.append(1 - prob_gone)
    
    df['probability_available'] = availability_probs
    df['vbd_score'] = df['salary_value']  # Use salary_value as VBD proxy
    
    return df

# Apply divergence metrics to our data
enhanced_df = add_divergence_metrics(ranking_df)

# Filter to top 225 players in either ESPN or ADP rankings
enhanced_df_filtered = enhanced_df[(enhanced_df['espn_rank'] <= 225) | (enhanced_df['adp_rank'] <= 225)].copy()

print("✅ Enhanced dataframe with divergence metrics created")
print(f"Total players before filtering: {len(enhanced_df)}")
print(f"Top 225 relevant players: {len(enhanced_df_filtered)}")
print(f"Divergence range: {enhanced_df_filtered['rank_divergence'].min():.1f} to {enhanced_df_filtered['rank_divergence'].max():.1f}")
print(f"Players with significant negative divergence (ESPN overvalued): {len(enhanced_df_filtered[enhanced_df_filtered['rank_divergence'] < -15])}")
print(f"Players with significant positive divergence (ESPN undervalued): {len(enhanced_df_filtered[enhanced_df_filtered['rank_divergence'] > 15])}")

✅ Enhanced dataframe with divergence metrics created
Total players before filtering: 300
Top 225 relevant players: 241
Divergence range: -123.0 to 92.0
Players with significant negative divergence (ESPN overvalued): 60
Players with significant positive divergence (ESPN undervalued): 46


## 2. Strategic Override Matrix

Four-quadrant analysis for override decisions based on divergence and availability.

In [13]:
def plot_override_matrix(df, my_picks=[8, 17, 32, 41]):
    """Four quadrant plot for override decisions"""
    
    # Focus on players relevant to next few picks
    next_pick = my_picks[0]
    relevant_players = df[df['adp_rank'] <= next_pick + 20].copy()
    
    fig = go.Figure()
    
    # Add scatter plot
    fig.add_trace(go.Scatter(
        x=relevant_players['rank_divergence'],
        y=relevant_players['probability_available'],
        mode='markers+text',
        text=[name[:8] + "..." if len(name) > 8 else name 
              for name in relevant_players['player_name']],
        textposition="top center",
        marker=dict(
            size=np.maximum(relevant_players['vbd_score']/4, 6),
            color=relevant_players['vbd_score'],
            colorscale='Viridis',
            showscale=True,
            colorbar=dict(title="VBD Score"),
            line=dict(width=1, color='black')
        ),
        hovertemplate='Player: %{text}<br>Divergence: %{x}<br>Availability: %{y:.1%}<br>VBD: %{marker.color}<extra></extra>'
    ))
    
    # Add quadrant lines
    fig.add_hline(y=0.5, line_dash="dash", line_color="gray", opacity=0.5)
    fig.add_vline(x=0, line_dash="dash", line_color="gray", opacity=0.5)
    
    # Add quadrant labels with strategic advice
    fig.add_annotation(
        x=-30, y=0.9,
        text="📈 ESPN OVERVALUED<br>🟢 LIKELY AVAILABLE<br>💡 Wait & Target Later",
        showarrow=False,
        bgcolor="lightgreen",
        bordercolor="green",
        borderwidth=1,
        font=dict(size=10)
    )
    
    fig.add_annotation(
        x=30, y=0.9,
        text="📉 ESPN UNDERVALUED<br>🟢 LIKELY AVAILABLE<br>🎯 Hidden Gems",
        showarrow=False,
        bgcolor="lightblue",
        bordercolor="blue",
        borderwidth=1,
        font=dict(size=10)
    )
    
    fig.add_annotation(
        x=-30, y=0.1,
        text="📈 ESPN OVERVALUED<br>🔴 WILL BE GONE<br>⚠️ Avoid - Others Overvaluing",
        showarrow=False,
        bgcolor="lightyellow",
        bordercolor="orange",
        borderwidth=1,
        font=dict(size=10)
    )
    
    fig.add_annotation(
        x=30, y=0.1,
        text="📉 ESPN UNDERVALUED<br>🔴 WILL BE GONE<br>🚨 Draft Now or Miss Out",
        showarrow=False,
        bgcolor="lightcoral",
        bordercolor="red",
        borderwidth=1,
        font=dict(size=10)
    )
    
    fig.update_layout(
        title=f'Strategic Override Decision Matrix - Top 225 Players (Next Pick: {next_pick})',
        xaxis_title='Rank Divergence (ESPN - ADP)',
        yaxis_title='Probability Available at Your Next Pick',
        height=700,
        width=1000,
        showlegend=False
    )
    
    return fig

# Create strategic override matrix
override_fig = plot_override_matrix(enhanced_df_filtered, my_picks=[8, 17, 32, 41])
override_fig.show()

print("\n📊 Strategic Override Matrix:")
print("🎯 Top Right (Blue): Hidden gems - ESPN undervalues, still available")
print("🚨 Bottom Right (Red): Urgent picks - ESPN undervalues, will be gone")  
print("🟢 Top Left (Green): Wait targets - ESPN overvalues, will be available")
print("⚠️  Bottom Left (Yellow): Avoid - ESPN overvalues, others will take anyway")


📊 Strategic Override Matrix:
🎯 Top Right (Blue): Hidden gems - ESPN undervalues, still available
🚨 Bottom Right (Red): Urgent picks - ESPN undervalues, will be gone
🟢 Top Left (Green): Wait targets - ESPN overvalues, will be available
⚠️  Bottom Left (Yellow): Avoid - ESPN overvalues, others will take anyway


## 3. Divergence Timeline

Bubble chart showing rank divergence across draft positions with VBD sizing.

In [14]:
def plot_divergence_timeline(df, current_pick=1):
    """Bubble chart of divergence with VBD sizing"""
    
    # Focus on players likely to be available in first few rounds
    df_plot = df[df['adp_rank'] <= 100].copy()
    df_plot = df_plot.sort_values('adp_rank')
    
    fig = go.Figure()
    
    # Create hover text with player details
    hover_text = [
        f"{row['player_name']}<br>" +
        f"Position: {row['position']}<br>" +
        f"ESPN Rank: {row['espn_rank']}<br>" +
        f"ADP Rank: {row['adp_rank']}<br>" +
        f"VBD Score: {row['vbd_score']}<br>" +
        f"Availability: {row['probability_available']:.1%}"
        for _, row in df_plot.iterrows()
    ]
    
    fig.add_trace(go.Scatter(
        x=df_plot['adp_rank'],
        y=df_plot['rank_divergence'],
        mode='markers',
        marker=dict(
            size=np.maximum(df_plot['vbd_score']/3, 5),  # Minimum size 5
            color=df_plot['rank_divergence'],
            colorscale='RdBu',
            cmid=0,
            showscale=True,
            colorbar=dict(title="Rank Divergence<br>(ESPN - ADP)"),
            line=dict(width=1, color='black')
        ),
        text=hover_text,
        hovertemplate='%{text}<extra></extra>',
        name='Players'
    ))
    
    # Add reference lines
    fig.add_hline(y=0, line_dash="dash", line_color="gray", opacity=0.5)
    fig.add_hline(y=15, line_dash="dot", line_color="red", opacity=0.7, 
                  annotation_text="Significant Divergence")
    fig.add_hline(y=-15, line_dash="dot", line_color="blue", opacity=0.7,
                  annotation_text="Significant Divergence")
    
    # Add user's pick positions
    my_picks = [8, 17, 32, 41, 56, 65, 80, 89]
    for pick in my_picks[:4]:  # First 4 picks
        fig.add_vline(x=pick, line_dash="dash", line_color="green", opacity=0.6,
                      annotation_text=f"Pick {pick}")
    
    fig.update_layout(
        title=f'Divergence vs Draft Position - Top 225 Players (Current Pick: {current_pick})',
        xaxis_title='ADP Rank (Draft Position)',
        yaxis_title='Rank Divergence (ESPN - ADP)',
        height=600,
        width=1000,
        showlegend=False
    )
    
    return fig

# Create divergence timeline
timeline_fig = plot_divergence_timeline(enhanced_df_filtered, current_pick=1)
timeline_fig.show()

print("\n📊 Divergence Timeline:")
print("- X-axis: Where players are typically drafted (ADP)")
print("- Y-axis: How much ESPN disagrees with ADP")
print("- Bubble size: VBD value")
print("- Red bubbles above line: ESPN undervalues (potential steals)")
print("- Blue bubbles below line: ESPN overvalues (potential traps)")
print("- Green lines: Your draft picks")


📊 Divergence Timeline:
- X-axis: Where players are typically drafted (ADP)
- Y-axis: How much ESPN disagrees with ADP
- Bubble size: VBD value
- Red bubbles above line: ESPN undervalues (potential steals)
- Blue bubbles below line: ESPN overvalues (potential traps)
- Green lines: Your draft picks


## 4. Divergence Summary Table

Concrete examples of the biggest blind spots for immediate actionable insights.

In [15]:
def create_divergence_summary_table(df, top_n=10):
    """Create summary table of biggest divergences with strategic advice - Top 225 players only"""
    
    # Get biggest positive and negative divergences from filtered data
    positive_divergence = df[df['rank_divergence'] > 0].nlargest(top_n, 'rank_divergence')
    negative_divergence = df[df['rank_divergence'] < 0].nsmallest(top_n, 'rank_divergence')
    
    print("🚨 BIGGEST BLIND SPOTS - ESPN SIGNIFICANTLY UNDERVALUES (Top 225 Players):")
    print("=" * 80)
    for _, player in positive_divergence.iterrows():
        advice = "🎯 TARGET" if player['probability_available'] > 0.7 else "🚨 URGENT"
        print(f"{advice} {player['player_name']:<20} | ESPN:{player['espn_rank']:>3.0f} ADP:{player['adp_rank']:>3.0f} | "
              f"Divergence: +{player['rank_divergence']:>2.0f} | Avail: {player['probability_available']:.1%}")
    
    print("\n📈 BIGGEST OVERVALUES - ESPN SIGNIFICANTLY OVERVALUES (Top 225 Players):")
    print("=" * 80)
    for _, player in negative_divergence.iterrows():
        advice = "⏳ WAIT" if player['probability_available'] > 0.7 else "⚠️ AVOID"
        print(f"{advice} {player['player_name']:<20} | ESPN:{player['espn_rank']:>3.0f} ADP:{player['adp_rank']:>3.0f} | "
              f"Divergence: {player['rank_divergence']:>3.0f} | Avail: {player['probability_available']:.1%}")
    
    # Position-specific insights
    print(f"\n📊 POSITION BREAKDOWN (Top 225 Players):")
    print("=" * 50)
    for pos in ['RB', 'WR', 'QB', 'TE']:
        pos_df = df[df['position'] == pos]
        if len(pos_df) > 0:
            avg_divergence = pos_df['rank_divergence'].mean()
            undervalued = len(pos_df[pos_df['rank_divergence'] > 15])
            overvalued = len(pos_df[pos_df['rank_divergence'] < -15])
            print(f"{pos}: Avg Divergence: {avg_divergence:+.1f} | Undervalued: {undervalued} | Overvalued: {overvalued}")

# Create and display summary table
create_divergence_summary_table(enhanced_df_filtered, top_n=8)

print("\n" + "=" * 80)
print("🎯 STRATEGIC TAKEAWAYS:")
print("- Focus on 🎯 TARGET and 🚨 URGENT players (positive divergence)")
print("- Consider waiting on ⏳ WAIT players (negative divergence, high availability)")
print("- Avoid reaching for ⚠️ AVOID players (negative divergence, low availability)")
print("- The 80% ESPN weight may cause you to miss significant value")
print("- Analysis filtered to top 225 players only for draft relevance")

🚨 BIGGEST BLIND SPOTS - ESPN SIGNIFICANTLY UNDERVALUES (Top 225 Players):
🎯 TARGET Jonnu Smith          | ESPN:206 ADP:114 | Divergence: +92 | Avail: 100.0%
🎯 TARGET Sam Darnold          | ESPN:267 ADP:175 | Divergence: +92 | Avail: 100.0%
🎯 TARGET Isaiah Likely        | ESPN:258 ADP:169 | Divergence: +89 | Avail: 100.0%
🎯 TARGET Aaron Rodgers        | ESPN:268 ADP:182 | Divergence: +86 | Avail: 100.0%
🎯 TARGET Rico Dowdle          | ESPN:222 ADP:142 | Divergence: +80 | Avail: 100.0%
🎯 TARGET Brashard Smith       | ESPN:282 ADP:205 | Divergence: +77 | Avail: 100.0%
🎯 TARGET Daniel Jones         | ESPN:296 ADP:222 | Divergence: +74 | Avail: 100.0%
🎯 TARGET Ja'Tavion Sanders    | ESPN:292 ADP:219 | Divergence: +73 | Avail: 100.0%

📈 BIGGEST OVERVALUES - ESPN SIGNIFICANTLY OVERVALUES (Top 225 Players):
⏳ WAIT Cairo Santos         | ESPN:190 ADP:313 | Divergence: -123 | Avail: 100.0%
⏳ WAIT Matt Gay             | ESPN:192 ADP:293 | Divergence: -101 | Avail: 100.0%
⏳ WAIT Brandin Cooks     

## 7. ESPN vs ADP Ranking Comparison

Direct comparison showing correlation and outliers between ranking sources.

In [16]:
def compare_ranking_sources(df, top_n=20):
    """Compare ESPN vs ADP rankings for top players - Top 225 only"""
    
    # Take top players by ADP
    top_players = df.nsmallest(top_n, 'adp_rank')
    
    fig = go.Figure()
    
    fig.add_trace(go.Scatter(
        x=top_players['espn_rank'],
        y=top_players['adp_rank'],
        mode='markers+text',
        text=top_players['player_name'],
        textposition="top center",
        marker=dict(size=10, color='blue'),
        name='Players'
    ))
    
    # Add diagonal line for perfect correlation
    max_rank = min(225, max(top_players['espn_rank'].max(), top_players['adp_rank'].max()))
    fig.add_trace(go.Scatter(
        x=[1, max_rank],
        y=[1, max_rank],
        mode='lines',
        line=dict(dash='dash', color='red'),
        name='Perfect Correlation'
    ))
    
    fig.update_layout(
        title='ESPN vs ADP Rankings Comparison (Top 225 Players)',
        xaxis_title='ESPN Rank',
        yaxis_title='ADP Rank',
        height=600,
        width=600
    )
    
    return fig

# Create ranking comparison
ranking_comparison = compare_ranking_sources(enhanced_df_filtered, top_n=20)
ranking_comparison.show()

print("\n📊 Ranking Comparison Analysis:")
print("- Points on diagonal = perfect agreement between ESPN and ADP")
print("- Points above line = ESPN ranks higher (player may be overvalued)")
print("- Points below line = ADP ranks higher (player may be undervalued)")
print("- Distance from line = magnitude of disagreement")


📊 Ranking Comparison Analysis:
- Points on diagonal = perfect agreement between ESPN and ADP
- Points above line = ESPN ranks higher (player may be overvalued)
- Points below line = ADP ranks higher (player may be undervalued)
- Distance from line = magnitude of disagreement


## Summary

These strategic divergence visualizations reveal critical blind spots in the 80% ESPN + 20% ADP system:

### Key Insights:
1. **Queue Jump Alerts** identify players likely to be drafted earlier than ESPN suggests
2. **Override Matrix** provides clear strategic guidance on when to override the system
3. **Divergence Timeline** shows where in the draft divergences are most significant
4. **Summary Table** gives concrete targets and avoids
5. **Blind Spot Radar** highlights ESPN's biggest undervaluations
6. **Consensus Analysis** reveals where ranking sources disagree most
7. **Direct Comparison** shows correlation patterns and outliers

### Strategic Application:
- Use these visualizations during draft preparation to identify value opportunities
- Override the 80/20 system when divergence exceeds reasonable thresholds
- Focus on players in the "urgent" quadrant of the Override Matrix
- Be cautious with players showing high ranking variance

### Analysis Scope:
- All visualizations filtered to only show top 225 players in either ESPN or ADP rankings
- This removes late-round irrelevant players from the analysis
- Focus on draft-relevant players for actionable insights